In [1]:
%load_ext Cython 
%pylab 
import numpy as np
from io import StringIO
import time
import threading 
import matplotlib.widgets as widgets
import random as pyrandom
def linlin(x, smi, sma, dmi, dma): return (x-smi)/(sma-smi)*(dma-dmi)+dmi
from threading import Thread
# from pyo import *
# Audio server
import pyo64 as pyo
from scipy.signal import butter, lfilter, freqz
# Is 1024 too large for the bufferSize in this case. 
fs = 44100/5
s = pyo.Server(sr=fs, nchnls=2, buffersize=1024, duplex=0).boot()
s.start()
# Need to adjust the maxsize to test the speed. 
fifo = pyo.FIFOPlayer(maxsize=20, mul=[.5,.5])
mixer = pyo.Mixer()
mixer.addInput(0, fifo);mixer.setAmp(0,0,1)
mixer.addInput(1, fifo);mixer.setAmp(1,1,1)
mixer.out()
fifo.out()



Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib

WxPython is not found for the current python version.
Pyo will use a minimal GUI toolkit written with Tkinter.
This toolkit has limited functionnalities and is no more
maintained or updated. If you want to use all of pyo's
GUI features, you should install WxPython, available here:
http://www.wxpython.org/



< Instance of FIFOPlayer class >

In [26]:
def proc(fifo, stopevent):
    global velSound, forceSound, which2play
    if (which2play == "velocity"):
        print "Play the velocity vector."
        fifo.put(velSound)
    if (which2play == "force"):
        print "Play the force vector"
        fifo.put(forceSound)
        
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def plotTrajectory(data, sigma, trj, resultWindow, audioVecSize):
        plotpot(data[:, 0:2], fig2 = resultWindow,  sigma = sigma )
        resultWindow.gca().plot(trj[:,1], trj[:,2], "-", lw=0.7, c = "green")
        resultWindow.gca().plot(trj[0,1], trj[0,2], "o",  c = "yellow")
        resultWindow.gca().plot(trj[audioVecSize - 1,1], trj[audioVecSize - 1,2], "x",  c = "red")
        resultWindow.gca().axis([-0.6, 0.6, -0.6, 0.6])
        resultWindow.canvas.draw()
    

# For winding
def makeWindow(s, rampUp = 0.02, rampDown = 0.02):
    w = np.append(np.linspace(0, 1.0, num = int(s * rampUp)), \
      np.ones(s - int(s * rampUp)- int(s * rampDown)))
    w = np.append(w, np.linspace(1.0, 0, num = int(s * rampDown)))
    return w

# Slider update function, for the interaction
def sliderUpdate(val):
    global dt, resistant
    sigma = ssigma.val
    exp_table = updateExpTable(sigma, delta_dis)
    dt = sdt.val
    resistant = sresistant.val
    
def sigmaUpdate(val):
    global sigma, exp_table, delta_dis
    sigma = ssigma.val
    exp_table = updateExpTable(sigma, delta_dis)
#-----------------------
    
# Create an exponential table for the lookup calculation of potential. 
def createExpTable( dim, sigma, exp_resolution = 1000000 ):
    delta_dis = np.linspace(0.0, dim, num = exp_resolution) # Range of distance difference. 
    tab = updateExpTable(sigma, delta_dis)
    return tab, dim

def updateExpTable(sigma, delta_dis):
    sigma2 = sigma * sigma
    # Try 2 time sigma2
    return exp(- delta_dis/(2 *sigma2))/sigma2
# ------------------------------------------------------#

Play the velocity vector.
Play the velocity vector.
Play the velocity vector.
Play the velocity vector.
Play the velocity vector.
Play the velocity vector.
Play the velocity vector.
Play the velocity vector.
Play the velocity vector.


In [30]:
# Currently this plotpot is only for interactive mode. 
# Eventually, data visualisation is not needed because it is aimed just for the sonification. 
# Data visualisation can be provided offline as a comparison. 
# For this method, visulisation is no use for higher dimension data anytaway 
def plotpot(data2d, fig2, sigma=.1, Nx=40, Ny=40):
    # ensure the data sending for plotpot is always in 2d. 
    potmap = zeros((Nx, Ny))
    # The potetnial pot should not be run everytime. 
    for i in range(Nx):
        for j in range(Ny):
            x = float(i)/Nx-0.5
            y = float(j)/Ny-0.5
            potmap[j,i] = potential_ds(data2d, np.array([x,y]), sigma)
    fig2.clf()
    matshow(potmap, cmap=cm.gray, extent=(-0.5,0.5,0.5,-0.5), fignum = 2)    
    fig2.gca().plot(data[:,0], data[:,1], ".")

def initializePot(data, N):  # It takes data and number of rows. 
    sctPlot = ax.scatter(data[:,0], data[:,1],  c="blue", picker = 2, s=[50]*N)
    fig.subplots_adjust(bottom=0.25, left=0.1)
    plt.grid(False)
    plt.axis([-0.6, 0.6, -0.6, 0.6])
    return sctPlot
    

def spectrum(av,fs):
    NFFT = 1024
    figure()
    specgram(av, NFFT = NFFT, Fs = fs, noverlap = 900, cmap= cm.gist_heat)
    show()
    
def stdData(data, dim):
    for i in range(dim):
        # S1 Standardize data. 
        data[:, i] = (data[:,i] - np.mean(data[:,i]))/np.std(data[:,1])
        # limit range to -.5 ~ .5 in each dimension
        data[:,i] = data[:,i] / np.max(np.absolute(data[:,i]))
        data[:,i] = data[:,i] / 2  
    return data

# Generate data set based of dimension and num_of_cluster. 
def dataGen(dim, c, sigma = 0.2, minnr = 50, maxnr = 200):
    for i in range(c):
        nr = pyrandom.randrange(minnr, maxnr, 1)
        meanvec = np.random.rand(dim)
        covmat = sigma**2*cov(np.random.rand(dim, dim))
        dtmp = np.random.multivariate_normal(meanvec, covmat, nr)
        if(i==0): 
            result = dtmp.copy() 
        else:
            result = np.vstack((result, dtmp.copy()))
    return stdData(result, dim)
    

In [4]:
%%cython
cimport numpy as np
import numpy as np
from libc.stdlib cimport rand, malloc, free
from libc.math cimport exp
from matplotlib import pyplot as plt
import time

def potential_ds(np.ndarray[np.float64_t, ndim = 2] data,\
                 np.ndarray[np.float64_t, ndim = 1] grid, double sigma=0.2):
    cdef int N, dim
    N, dim = data.shape[0], data.shape[1]
    cdef double potential, p_sum = 0.0
    cdef int j, i
    cdef double sigma2 = sigma *sigma 
    
    for j in range(N):
        for i in range(dim):
            p_sum += (grid[i] - data[j,i]) * (grid[i] - data[j,i])
            
        potential += -exp (-0.5*p_sum / sigma2)
        p_sum = 0
    return potential
    
# put pos outside and make it a np array. 
def PTSM(np.ndarray[np.float64_t, ndim = 1] pos, np.ndarray[np.float64_t, ndim = 2] X,\
         np.ndarray[np.float64_t, ndim = 1] vel, np.ndarray[np.float64_t, ndim = 1] lookupExp,\
         int table_res, double norm_max, double sigma=0.25,\
         double dt=0.01, double r=0.99, int Nsamp=5000, double compensation = 0.01 ):
    cdef int N, dim
    N, dim = X.shape[0], X.shape[1]
    cdef int i,j, steps,lookupIdx  # These 3 are for iterations. 
    cdef double sigma2,m
    cdef double d,V, v_sum = 0.0
    cdef double force1p_sum = 0.0
    cdef double vel_sum = 0.0
    cdef double *force = <double *>malloc(dim * sizeof(double))
    cdef double *trj = <double *>malloc(Nsamp * (dim+ 1) *sizeof(double))
    cdef double *velocity = <double *>malloc(dim * sizeof(double))
    cdef double *position = <double *>malloc(dim * sizeof(double))
    cdef double *temp = <double *>malloc(dim * sizeof(double)) 
    
    cdef double *viewForce = <double *>malloc(Nsamp* dim * sizeof(double))
#     cdef double testing = 0.0

    for i in range(dim):
        velocity[i] = vel[i]
        position[i] = pos[i]
        
    sigma2 = sigma * sigma
    m = compensation/sigma2
    #--------------------
       # Force
    for steps in range(Nsamp): 
        for i in range(dim):
            force[i] = 0
        for j in range (N):
            for i in range (dim):
                temp[i] = (position[i] - X[j,i])
                force1p_sum  += temp[i] * temp[i]
            # Look for exp index 
            
            lookupIdx = int(force1p_sum/norm_max * table_res )
#             print lookupIdx
            # Prevent index become much bigger than the table 
            if (lookupIdx >= table_res):
                lookupIdx = table_res - 1
            for i in range (dim):  
#                 print lookupExp[lookupIdx]
                force[i] += -temp[i] * lookupExp[lookupIdx]
               
            force1p_sum = 0
    #------------
    #         Now update pos and vel 
    #         This is the main part to get the trajectory information in terms of the new velocity 
    #         and position
        
        for i in range(dim):
            viewForce[steps*dim + i] = force[i]
            # Should be dt /m * force[i], but since m = 1.0
            velocity[i] = r * velocity[i] + dt * force[i] / m 
            position[i] = position[i] + dt * velocity[i]
            vel_sum += velocity[i] 

            #-----------------------  
    #         # Put velocity and new position into the trj array. 
        for i in range(dim + 1):
            if i == 0: # The first column is always given to the velocity
                trj[steps * (dim + 1) + i] =  vel_sum
            else:
                trj[steps * (dim + 1) + i] = position[i - 1]
        vel_sum = 0
#     # put trj into a numpy array. Because you can't return a c-array.              
    resultTrj = np.zeros(Nsamp * (dim + 1), dtype = np.float64)
    resultVel = np.zeros(dim)
    resultForce = np.zeros(Nsamp * dim)
    # Move c array to numpy array for return 
    for i in range(Nsamp* (dim + 1)):
        resultTrj[i] = trj[i]     
    for i in range(dim): 
        resultVel[i] = velocity[i]  

    for i in range(dim * Nsamp):
        resultForce[i] = viewForce[i]
    # Transpose matrix
    resultTrj = np.reshape(resultTrj, (-1, dim + 1))  # Correct the matrix shape. 
#     # Free up memories. 
#     print testing
    free(trj)
    free(force)
    free(velocity)
    free(position)
    free(viewForce)
    return resultTrj, resultVel, resultForce


In [23]:

# if testMode == T, a random example is used as the data. 
# else, you can load the actual data txt files. 
testMode = True
if testMode :
    data = dataGen(dim = 4, c = 3,sigma = 0.3, minnr = 100, maxnr = 300)
    N, dim = data.shape[0], data.shape[1]
    pylab.rcParams['figure.figsize'] = (8.0, 8.0)
    ct=1
    plot(data[:, 0], data[:,1], ".")
#     for i in range(dim):
#         for j in range(dim):
#             subplot(dim, dim, ct)
#             plot(data[:, i], data[:,j], ".")   
#             ct += 1  

else:
    # Load test data from file 
#     data = np.loadtxt("data/dim032.txt")
    # Still have problem loading csv file. 
    data = np.genfromtxt("data/posture.csv",delimiter=",", skip_header = 1)
    headers = np.genfromtxt("data/posture.csv", dtype = np.str_, delimiter = ",", max_rows = 1 )
    N, dim = data.shape[0], data.shape[1]
#     data = stdData(data, dim)
#     plot(data[:,0], data[:,1], ".")

# Create exp lookup
exp_resolution = 1000000 # Resolution for lookup table
delta_dis = np.linspace(0.0, dim, num = exp_resolution) # Range of distance difference. 
# m_comp = 0.001 # mass compensation
norm_max = dim
sigma = 0.1
exp_table = updateExpTable(sigma, delta_dis)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:47: RuntimeWarning: covariance is not positive-semidefinite.


In [24]:
class soundOption(object):
    
    def velocity(self, event):
        global which2play
        which2play = "velocity"

    
    def force(self, event):
        global which2play
        which2play = "force"

## Create loop up table for exp()


## Interaction here. 

Offer visualise mode and only sound mode. 

To do list:
 
3. OpenCL 
4. Benchmarking
5. Hardware. 
4. Multiple particle mode 


In [28]:
fig, ax = plt.subplots()

m_comp, dt, resistant, sigma = 0.02, 0.005 , 0.999, 0.15 # init 
max_sigma = log(dim)/2.5 * 0.5 + 0.1

t = 1.0 # Time in second per piece
blockSize = 5000   # Buffer size for trajectory
audioVecSize  = int(t *  blockSize)  # I define that 5000 steps will return 1 second of audio 
exp_table, norm_max = createExpTable(dim, sigma, exp_resolution = exp_resolution)
windowing = makeWindow(audioVecSize, rampUp = 0.05, rampDown = 0.05) # Windowing for audio
velSound = np.zeros(audioVecSize)
forceSound = np.zeros(dim*audioVecSize) # This is only meant for visualisation. 
drawResult = True
which2play = "velocity" # Initialise the button press
#-------------------------------------------------#

# Initialize the plot windows. 
sctPlot = initializePot(data, N)

def on_pick(event):
    # In the continuous mode, window shouldn't be used. 
    global data, sigma, resistant, dt, resultWindow, windowing, audioVecSize,\
    norm_max, velSound, drawResult, dim, forceSound, fs
    vel = (rand(dim) - 0.5) 
    # Initialise velocity. 
    
    artist = event.artist # Current click event
    ind = np.array(event.ind) # Get the index of the clicked data
    pos = np.array(data[ind[0],:])
    
    # Get PTSM trajectory information
    trj, junk, forceSound = PTSM(pos, data, vel, exp_table, exp_resolution, \
                norm_max, sigma = sigma, dt = dt, r = resistant,\
                Nsamp = audioVecSize, compensation = m_comp)
    #-------------------------------------------#
    # Process sound #----------
    velSound = trj[:,0] / np.max(np.absolute(trj[:,0]))* windowing 
    velSound = butter_lowpass_filter(velSound, 2000.0, fs, 6) # 6th order
    forceSound = forceSound/np.max(np.absolute(forceSound))    
    stopevent = threading.Event()
    producer = threading.Thread(name="Compute audio signal", target=proc, args=[fifo, stopevent])
    producer.start() 
    if drawResult == True:
#         plotTrajectory(data = data, sigma = sigma, trj = trj, 
#                        resultWindow = resultWindow, audioVecSize = resultWindow)
        plotpot(data[:, 0:2], fig2 = resultWindow,  sigma = sigma )
        resultWindow.gca().plot(trj[:,1], trj[:,2], "-", lw=0.7, c = "green")
        resultWindow.gca().plot(trj[0,1], trj[0,2], "o",  c = "yellow")
        resultWindow.gca().plot(trj[audioVecSize - 1,1], trj[audioVecSize - 1,2], "x",  c = "red")
        resultWindow.gca().axis([-0.6, 0.6, -0.6, 0.6])
        resultWindow.canvas.draw()


# ---------------#
# Create a slider for setting up the velocity
axcolor = 'lightgoldenrodyellow'
# Create sliders for sigma and dt. 
axSigma = plt.axes([0.1, 0.1, 0.8, 0.02], axisbg = axcolor)
axDt = plt.axes([0.1, 0.06, 0.8, 0.02], axisbg = axcolor)
axR = plt.axes([0.1, 0.02, 0.8, 0.02], axisbg = axcolor)
axB1 = plt.axes([0.1, 0.13, 0.1, 0.075], axisbg = axcolor)
axB2 = plt.axes([0.21, 0.13, 0.1, 0.075], axisbg = axcolor)

ssigma = Slider(axSigma, "Sigma", 0.001, max_sigma, valinit=sigma, color='blue')
sdt = Slider(axDt, "dt", 0.001, 0.01, valinit = dt, color = 'blue')
sresistant = Slider(axR, "r", 0.99, 1.0, valinit = resistant, color = 'blue')

choice = soundOption()

b1 = Button(axB1, "Velocity")
b1.on_clicked(choice.velocity)
b2 = Button(axB2, "Force")
b2.on_clicked(choice.force)


ssigma.on_changed(sigmaUpdate)
sdt.on_changed(sliderUpdate)
sresistant.on_changed(sliderUpdate)
resultWindow = figure(2, figsize=(8, 8))
fig.canvas.mpl_connect('pick_event', on_pick)



Play the velocity vector.


21

In [ ]:
figure()
plot(force)

In [ ]:
%pylab

In [ ]:
max_sigma

In [ ]:
force = force/np.max(np.absolute(force))

In [ ]:
np.max(np.absolute(force))

In [34]:
spectrum(velSound,fs)